In [2]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict, ChainMap, Counter
import glob
import sys 
import functools
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import dask.dataframe as dd
from multiprocessing import Pool, Manager, cpu_count
from joblib import Parallel, delayed
import pymp
import gezi
from gezi import tqdm
tqdm.pandas()

In [3]:
timer = gezi.Timer('read user_action2.feather', True)
d = pd.read_feather('../input/user_action2.feather')
d.feedid = d.feedid.astype(int)
d.userid = d.userid.astype(int)
d.date_ = d.date_.astype(int)
timer.print()

read user_action2.feather start
read user_action2.feather duration: 1.0604383945465088


In [4]:
d = d.sort_values(['date_'], ascending=True)

In [5]:
d.head()

,index,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,version,finish_rate,stay_rate,is_first,actions
0,0,8,71474,1,1,0,0,1,500,5366,0,0,0,0,2,0.045455,0.487818,1,1
319000,6957070,237907,3404,1,2,0,0,0,13573,13871,0,0,0,0,2,0.226217,0.231183,1,0
318999,6957069,237907,87814,1,2,0,0,0,0,333,0,0,0,0,2,0.000000,0.005550,1,0
318998,6957068,237907,58723,1,2,0,0,0,0,1973,0,0,0,0,2,0.000000,0.078920,1,0
318997,2435793,82352,69580,1,2,0,0,0,13241,13496,0,0,0,0,2,1.203727,1.226909,1,0


In [7]:
len(set(d[d.date_ == 14].feedid) - set(d[d.date_ < 14].feedid))

4363

In [8]:
len(set(d[d.date_ == 14].feedid) - set(d[d.date_ < 14].feedid)) / len(set(d[d.date_ == 14].feedid))

0.10316371890664901

In [14]:
x = len((set(d[(d.date_ == 14) & (d.userid % 2 == 1)].feedid) - set(pd.concat([d[(d.date_ == 14) & (d.userid % 2 == 0)], d[d.date_ < 14]]).feedid)) )
x

616

In [15]:
x / len(set(d[(d.date_ == 14) & (d.userid % 2 == 1)].feedid))

0.01800327332242226

In [17]:
x2 = len((set(d[(d.date_ == 14) & (d.userid % 2 == 1)].feedid) - set(d[d.date_ < 14].feedid)))
x2

3744

In [18]:
len(set(d.feedid))

96564

In [19]:
num_feedids = len(open('../input/feed_embeddings.csv').readlines()) - 1
num_feedids

106444

In [20]:
len(set(d.feedid)) / num_feedids

0.907181240840254

In [24]:
da = pd.read_csv('../input/test_a.csv')

In [25]:
len(set(da.feedid) - set(d.feedid)) / len(set(da.feedid))

0.07415308473419234

In [26]:
db = pd.read_csv('../input/test_b.csv')

In [27]:
len(set(db.feedid) - set(d.feedid)) / len(set(db.feedid))

0.07587018278860226

In [29]:
len(set(db.feedid) - (set(d.feedid) | set(da.feedid))) / len(set(db.feedid))

0.007145111767912996

In [30]:
len(set(d.feedid) | set(da.feedid) | set(db.feedid)) / num_feedids

0.9340122505730712